# Modelling a Pandemic

The following notebook contains visualizations to model the ongoing Covid-19 pandemic for my semester project in the module Data Visualization.

## Todo:

- Add maths and reproduction number
- Add different kinds of graphs

## Importing libraries

In [94]:
import pandas as pd
import numpy as np
import scipy
from scipy.integrate import odeint

from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, HoverTool, CDSView, CustomJS, Div, Slider, Spinner, Select, DataTable, TableColumn
from bokeh.layouts import column, layout, row
from bokeh.models.widgets import Tabs, Panel

from ipywidgets import interact

import plotly as plt
import plotly.express as px

#from sklearn import preprocessing
#https://discourse.bokeh.org/t/how-to-get-a-drop-down-to-change-axes-i-thought-this-would-work-but-i-dont-understand-why-not/4522/4

# Import data
The data used for this notebook is provided by Our World in Data. 

Our World in Data [Github](https://github.com/owid/covid-19-data/tree/master/public/data)

In [95]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv"
data = pd.read_csv(url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   iso_code                                    234 non-null    object 
 1   continent                                   221 non-null    object 
 2   location                                    234 non-null    object 
 3   last_updated_date                           234 non-null    object 
 4   total_cases                                 229 non-null    float64
 5   new_cases                                   229 non-null    float64
 6   new_cases_smoothed                          229 non-null    float64
 7   total_deaths                                223 non-null    float64
 8   new_deaths                                  223 non-null    float64
 9   new_deaths_smoothed                         223 non-null    float64
 10  total_cases_pe

In [96]:
df = data[['continent', 'location','last_updated_date', 'new_cases', 'new_deaths', 'total_cases', 'total_deaths', 'total_cases_per_million', 'total_deaths_per_million', 'people_fully_vaccinated','total_vaccinations_per_hundred', 'reproduction_rate', 'population_density','population','median_age','life_expectancy','gdp_per_capita', 'human_development_index', 'extreme_poverty']]
df = df[df['continent'].notna()]
df['total_cases_in_100'] = df['total_cases'] / 100
df['total_deaths_in_100'] = df['total_deaths'] / 100
df

,continent,location,last_updated_date,new_cases,new_deaths,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million,people_fully_vaccinated,...,reproduction_rate,population_density,population,median_age,life_expectancy,gdp_per_capita,human_development_index,extreme_poverty,total_cases_in_100,total_deaths_in_100
0,Asia,Afghanistan,2022-06-04,31.0,0.0,180615.0,7708.0,4534.029,193.496,4807917.0,...,NaN,54.422,39835428.0,18.6,64.83,1803.987,0.511,NaN,1806.15,77.08
2,Europe,Albania,2022-06-04,32.0,0.0,276342.0,3497.0,96188.078,1217.223,1239990.0,...,NaN,104.871,2872934.0,38.0,78.57,11803.431,0.795,1.1,2763.42,34.97
3,Africa,Algeria,2022-06-04,0.0,0.0,265889.0,6875.0,5959.415,154.091,NaN,...,NaN,17.348,44616626.0,29.1,76.88,13913.839,0.748,0.5,2658.89,68.75
4,Europe,Andorra,2022-06-04,173.0,0.0,43067.0,153.0,556752.075,1977.920,53442.0,...,NaN,163.755,77354.0,NaN,83.73,NaN,0.868,NaN,430.67,1.53
5,Africa,Angola,2022-06-04,234.0,0.0,99761.0,1900.0,2939.888,55.992,6555241.0,...,NaN,23.890,33933611.0,16.8,61.15,5819.495,0.581,NaN,997.61,19.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Asia,Vietnam,2022-06-04,881.0,0.0,10724554.0,43080.0,109246.021,438.836,79373469.0,...,NaN,308.127,98168829.0,32.6,75.40,6171.884,0.704,2.0,107245.54,430.80
229,Oceania,Wallis and Futuna,2022-06-04,0.0,0.0,454.0,7.0,40923.021,630.972,NaN,...,NaN,NaN,11094.0,NaN,79.94,NaN,NaN,NaN,4.54,0.07
231,Asia,Yemen,2022-06-04,0.0,0.0,11822.0,2149.0,387.726,70.481,429721.0,...,NaN,53.508,30490639.0,20.3,66.12,1479.147,0.470,18.8,118.22,21.49
232,Africa,Zambia,2022-06-04,292.0,0.0,322207.0,3988.0,17029.377,210.775,3674839.0,...,NaN,22.995,18920657.0,17.7,63.89,3689.251,0.584,57.5,3222.07,39.88


# Visualizing the current situation globally:
Even though most Covid-19 restrictions are lowered globally, there are still new infections and deaths every day. While rich countries are mostly fully vaccinated, countries in Africa or South America are still lacking sufficient healthcare. Especially, since poorer countries are not able to inform and test their citizens.

In [97]:
globe_data = data
globe_data = globe_data[globe_data['continent'].notna()]
#Add a normalized new_cases
start = 1
end = 10
width = end - start
globe_data['new_cases_norm'] = (globe_data['new_cases'] - globe_data['new_cases'].min())/(globe_data['new_cases'].max() - globe_data['new_cases'].min()) * width + start
globe_data['new_cases_norm'][np.isnan(globe_data['new_cases_norm'])] = 1
globe_data

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_34224/1250274219.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_34224/1250274219.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,new_cases_norm
0,AFG,Asia,Afghanistan,2022-06-04,180615.0,31.0,70.429,7708.0,0.0,1.000,...,NaN,37.746,0.50,64.83,0.511,NaN,NaN,NaN,NaN,1.004095
2,ALB,Europe,Albania,2022-06-04,276342.0,32.0,37.286,3497.0,0.0,0.000,...,51.2,NaN,2.89,78.57,0.795,NaN,NaN,NaN,NaN,1.004227
3,DZA,Africa,Algeria,2022-06-04,265889.0,0.0,2.286,6875.0,0.0,0.000,...,30.4,83.741,1.90,76.88,0.748,NaN,NaN,NaN,NaN,1.000000
4,AND,Europe,Andorra,2022-06-04,43067.0,173.0,24.714,153.0,0.0,0.000,...,37.8,NaN,NaN,83.73,0.868,NaN,NaN,NaN,NaN,1.022854
5,AGO,Africa,Angola,2022-06-04,99761.0,234.0,67.714,1900.0,0.0,0.000,...,NaN,26.664,NaN,61.15,0.581,NaN,NaN,NaN,NaN,1.030912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,VNM,Asia,Vietnam,2022-06-04,10724554.0,881.0,1170.429,43080.0,0.0,0.286,...,45.9,85.847,2.60,75.40,0.704,NaN,NaN,NaN,NaN,1.116384
229,WLF,Oceania,Wallis and Futuna,2022-06-04,454.0,0.0,0.000,7.0,0.0,0.000,...,NaN,NaN,NaN,79.94,NaN,NaN,NaN,NaN,NaN,1.000000
231,YEM,Asia,Yemen,2022-06-04,11822.0,0.0,0.429,2149.0,0.0,0.000,...,29.2,49.542,0.70,66.12,0.470,NaN,NaN,NaN,NaN,1.000000
232,ZMB,Africa,Zambia,2022-06-04,322207.0,292.0,100.571,3988.0,0.0,0.429,...,24.7,13.938,2.00,63.89,0.584,NaN,NaN,NaN,NaN,1.038574


In [98]:
map_figure = px.scatter_geo(globe_data,
                            locations = 'iso_code',
                            projection = 'orthographic',
                            color = 'continent',
                            size = 'new_cases_norm',
                            hover_name = 'location',
                            hover_data = ['new_cases','total_cases', 'total_deaths'])
map_figure.show()

# Preparing the SIR Model

In [99]:
def SIR(y, t, beta, gamma):
    S, I, R = y
    dS_dt = -beta * S * I
    dI_dT = beta * S * I - gamma * I
    dR_dt = gamma * I
    return([dS_dt, dI_dT, dR_dt])

def inputs(S0, I0, R0, beta, gamma):
    return S0, I0, R0, beta, gamma

def solver(SIR, inputs):
    S0 = inputs[0]
    I0 = inputs[1]
    R0 = inputs[2]
    beta = inputs[3]
    gamma = inputs[4]
    t = np.linspace(0, 365, 365)
    data = scipy.integrate.odeint(SIR, [S0, I0, R0], t,
                                  args = (beta, gamma))
    data = np.array(data)
    #data = ColumnDataSource(data = dict(S0 = S0, I0 = I0, R0 = R0, t = t
    #                                   , beta = beta, gamma = gamma))
    return data


def sourceInput(S, I, R, b, g):
    data = solver(SIR, inputs(S, I, R, b, g))
    data = pd.DataFrame(data)
    data = data.rename(columns = {0: "Susceptible", 1: "Infected", 
                                  2: "Recovered"})
    data["Days"] = (data.index -1)+1
    return data

# Preparing owid data

### Normalizing the population

In [100]:
start = 5
end = 10
width = end - start
df['pop_norm'] = (df['population'] - df['population'].min())/(df['population'].max() - df['population'].min()) * width + start

var_selection = ['total_cases', 'total_deaths', 'total_cases_per_million', 'total_deaths_per_million', 'people_fully_vaccinated', 'total_vaccinations_per_hundred', 'reproduction_rate', 'population_density', 'population', 'median_age', 'life_expectancy','gdp_per_capita', 'human_development_index', 'total_cases_in_100', 'total_deaths_in_100']

## Plotting SIR in Bokeh

In [103]:
#SIR visualization
sirDf = (sourceInput(0.999,0.001, 0, 0.15, 0.05))
sirDf['Susceptible'] = sirDf['Susceptible'] * 1000
sirDf['Susceptible'] = round(sirDf['Susceptible'])
sirDf['Infected'] = sirDf['Infected'] * 1000
sirDf['Infected'] = round(sirDf['Infected'])
sirDf['Recovered'] = sirDf['Recovered'] * 1000
sirDf['Recovered'] = round(sirDf['Recovered'])
sirDf = ColumnDataSource(sirDf)
#view = CDSView(source=source)

#Figure
p2 = figure(title = 'SIR-Model', x_axis_label='Days', y_axis_label = 'Population')

#Hovertool for more information on the SIR model
hover_tool = HoverTool(tooltips =  [("Susceptible", "@Susceptible")
                                   , ("Infected", "@Infected")
                                   , ("Recovered", "@Recovered")
                                   , ("Days", "@Days")
                                   ,])
p2.add_tools(hover_tool)

#Plot SIR
p2.line(x = 'Days', y = 'Susceptible', source = sirDf, color = 'Blue')
p2.line(x = 'Days', y = 'Infected', source = sirDf, color = 'Red')
p2.line(x = 'Days', y = 'Recovered', source = sirDf, color = 'Green')

#Add Dataframe to the dashboard
columns = [
        TableColumn(field="Susceptible", title="Susceptible"),
        TableColumn(field="Infected", title="Infected"),
        TableColumn(field = "Recovered", title = "Recovered"),
        TableColumn(field = "Days", title= "Days")
    ]

data_table = DataTable(source = sirDf, columns = columns, width = 500,
                      height = 280,)

div = Div(text=r"""
<b> A plot describing a SIR Model to plot an epidemic.</b> </p>
The amount of susceptible people <b>(S)</b> at time t
$$ \frac{dS}{dt} = \mu * (N - S) - \beta *I * \frac{S}{N} $$
<p\>
The amount of infected people <b>(I)</b> at time t 
<p\>
$$ \frac{dI}{dt} = \beta * I * \frac{S}{N} - (\mu + \gamma) * I $$
<p\>
The amount of Recovered people <b>(R)</b> at time t
$$\frac{dR}{dt} = \gamma * I - \mu * R $\ $$
<p\>
<p>
<p> $$N$$ describing the total amount of indiviuums.</p>
<b> $$\beta$$ </b> describing the transmissionrate of the virus.</b> </p>
<b> $$\gamma$$ </b> describing the recovery time of an individuum.</b>
""")

#Create the layout
sir_Layout = layout([[p2], [data_table, div]])
tab2 = Panel(child = sir_Layout, title = "SIR model")

# Plotting owid data

In [104]:
#Create active axis for visualizing owid data
df['active_axisX'] = df['total_cases_in_100']
df['active_axisY'] = df['total_deaths_in_100']
owidSource = ColumnDataSource(df)
view = CDSView(source=owidSource)

#Dashboard figure
p = figure(title = 'Covid-19 Dashboard', x_axis_label ="x", y_axis_label = "y")

#Axesselect for x-Axis
Axesselect = Select(title="Choose x-Axis variable:", value="total_cases_in_100", options= var_selection)
Axesselect.js_on_change('value', CustomJS(args=dict(source=owidSource, Axesselect=Axesselect, xaxis=p.xaxis), code="""
    source.data['active_axisX'] = source.data[Axesselect.value]
    source.change.emit()
    xaxis.axis_label = Axesselect.value
    """))

YAxesselect = Select(title="Choose y-Axis Variable:", value="total_deaths_in_100", options= var_selection)
YAxesselect.js_on_change('value', CustomJS(args=dict(source=owidSource, YAxesselect=YAxesselect, yaxis=p.yaxis), code="""
    source.data['active_axisY'] = source.data[YAxesselect.value]
    source.change.emit()
    Yaxis.axis_label = Axesselect.value
    """))

#Add hovertools for more information
hover_tool = HoverTool(tooltips =  [("Location", "@location")
                                   , ("Continent", "@continent")
                                   , ("Reproduction Rate", "@reproduction_rate")
                                   , ("Total vaccinations per 100", "@total_vaccinations_per_hundred")
                                   , ("Population Density", "@population_density")
                                   ,])
p.add_tools(hover_tool)

#Add color
index_cmap = factor_cmap('continent', palette=['red', 'blue', 'green', 'yellow', 'black', 'orange'], 
                         factors=sorted(df['continent'].unique()))
#Add data to the figure
p.circle(x = 'active_axisX', y = 'active_axisY', source=owidSource, view=view, size = 'pop_norm'
        , fill_color = index_cmap, legend_label = 'continent')

#Add the dropdown menu
controls = [YAxesselect, Axesselect]
inputs = column(*controls, width = 0)

owid_Layout = layout([[inputs, p]], sizing_mode= "scale_height")#scale_width
tab1 = Panel(child= owid_Layout, title="Covid-19 Dashboard")

#Create the full layout
tabs = Tabs(tabs=[tab1, tab2])
show(tabs)

### Code junk

In [ ]:
#Select
#Axesselect = Select(title="Country size:", value= 'population', options= populationData['location'].tolist()) 
#Axesselect.js_on_change('value', CustomJS(args=dict(source=pop_source, Axesselect=Axesselect, yaxis=p2.yaxis), code="""
#    sir_source.data['Susceptible'] = sir_source.data['Susceptible'] * 10
#    sir_source.data['I_Pop'] = sir_source.data['Infected'] * source.data[Axesselect.value]
#    sir_source.data['R_Pop'] = sir_source.data['Recovered'] * source.data[axecelect.value]
#    sir_source.change.emit()
#    """))

#Sliders
#betaSlider = Slider(title ="Beta", value = 0.5, start = 0, end = 1, step = 0.01)
#gammaSlider = Slider(title = "Gamma", value = 0.2, start = 0, end = 1, step = 0.01)

#popCallback = CustomJS(args = dict(source = source, pop = populationSlider),
#                  code = """
#                  const pop = pop.value;
#                  const s = source.data['Susceptible'];
#                  const inf = source.data['Infected'];
#                  const r = source.data['Recovered'];
#                  for(let i = 0; i < s.length; i++){
#                      s[i] = 10;
#                      inf[i] = inf[i] * pop
#                      r[i] = r[i] * pop
#                  }
#                  source.change.emit();
#""")

#callback = CustomJS(args=dict(source=source),
#                    code="""
#    const data = source.data;
#    var s = data['Susceptible'];
#    var inf = data['Infected'];
#    var r = data['Recovered']
#    var p = cb_obj.value;
#    for (let i = 0; i < s.length; i++) {
#        s[i] = s[i] * p;
#        inf[i] = inf[i] * p;
#        r[i] = r[i] * p;
#    }
#    source.change.emit();
#""")
#populationSlider = Slider(title = "Population size", value = 1000, start = 1, end = 100000, step = 100)
#populationSlider.js_on_change('value', callback)

#callback = CustomJS(args = dict(source = source, beta = betaSlider, gamma = gammaSlider),
#                   code = """
#                   const data = source.data;
#                   const beta = beta.value;
#                   const gamma = gamma.value;
#                   const S = source.data['Susceptible'];
#                   const I = source.data['Infected'];
#                   const R = source.data['Recovered'];
#                   const source.data  = solver(SIR, inputs(S, I, R, beta, gamma))
#                   source.change.emit();
#""") 
#betaSlider.js_on_change('value', callback)
#gammaSlider.js_on_change('value', callback)


#Add the dropdown
#controls = [populationSlider]#Axesselect, betaSlider, gammaSlider]
#inputs = column(*controls, width=0)